In [2]:
list(range(0))

[]

In [3]:
list(range(1))

[0]

In [4]:
import argparse
argparser = argparse.ArgumentParser()
argparser.add_argument("--mojo",nargs="+")

_StoreAction(option_strings=['--mojo'], dest='mojo', nargs='+', const=None, default=None, type=None, choices=None, help=None, metavar=None)

In [5]:
from torch_geometric import datasets

In [6]:
karat = datasets.KarateClub()
karat.data

/data01/sunyifan/resources/miniconda3/envs/mygcn/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34])

In [7]:
gnb = datasets.GNNBenchmarkDataset(root='/data01/sunyifan/work_station/my_gcn/data',name='CIFAR10')

In [8]:
gnb.len()


45000

In [9]:
root='/data01/sunyifan/work_station/my_gcn/data'
amazon = datasets.Amazon(root=root,name="Photo")
mutag = datasets.TUDataset(root=root,name="IMDB-BINARY")
mutag.len()

1000

In [10]:
mutag[0]

Data(edge_index=[2, 146], y=[1], num_nodes=20)

In [11]:
amazon.len()

1

In [12]:
amazon[0]

Data(x=[7650, 745], edge_index=[2, 238162], y=[7650])

In [32]:
import torch
indices = torch.tensor(range(7650))

In [33]:
amazon[0].subgraph(indices)

Data(x=[7650, 745], edge_index=[2, 238162], y=[7650])

In [15]:
amazon.num_classes

8

In [16]:
amazon.num_node_features

745

In [17]:
cora = datasets.Planetoid(root=root,name="Cora")
cora.len()

1

In [18]:
cora[0].y

tensor([3, 4, 4,  ..., 3, 3, 3])

In [19]:
from torch_geometric.utils import *
one_hot(cora[0].y).shape

torch.Size([2708, 7])

In [20]:
cora.num_classes

7

In [21]:
cora[0].edge_index.type()

'torch.LongTensor'

In [22]:
cora[0].val_mask

tensor([False, False, False,  ..., False, False, False])

In [23]:
cora[0].x.shape[0]

2708

In [24]:

def get_mask_ratio(data,ratio,rnd=None):
    """
    固定种子生成mask
    """
    j = 0
    for i in ratio:
        j+=i
    assert j==1 and len(ratio)==3
    # train,test,val
    num_nodes = data.x.shape[0]
    masks = []
    for i in ratio:
        masks.append(int(num_nodes*i))
    if sum(masks)!=num_nodes:
        masks[-1]=num_nodes-masks[0]-masks[1]
    rnd_index = []
    for i in range(num_nodes):
        rnd_index.append(i)
    if rnd:
        random.seed(rnd)
        random.shuffle(rnd_index)
    train_d,test_d,val_d = rnd_index[:masks[0]],rnd_index[masks[0]:masks[0]+masks[1]],rnd_index[masks[0]+masks[1]:]
    print(f"Splitting {num_nodes} nodes into {masks}")
    train_mask= torch.BoolTensor([False]*num_nodes)
    test_mask= torch.BoolTensor([False]*num_nodes)
    val_mask = torch.BoolTensor([False]*num_nodes)
    for i in train_d:
        train_mask[i] = True
    for i in test_d:
        test_mask[i] = True
    for i in val_d:
        val_mask[i] = True
    return (train_mask,test_mask,val_mask)
 

In [25]:
import random,torch
temp = get_mask_ratio(amazon[0],(0.7,0.12,0.18),1)
amazon[0].test_mask = temp[0]
amazon[0]

Splitting 7650 nodes into [5355, 918, 1377]


Data(x=[7650, 745], edge_index=[2, 238162], y=[7650])

In [26]:
from torch_geometric import transforms as T
transform = T.RandomNodeSplit(split="train_rest",num_val=100,num_test=100)
data = amazon[0]
data = transform(data)
data


Data(x=[7650, 745], edge_index=[2, 238162], y=[7650], train_mask=[7650], val_mask=[7650], test_mask=[7650])

In [27]:
x = amazon.copy()
x[0]

Data(x=[7650, 745], edge_index=[2, 238162], y=[7650])

In [28]:
def node_split(data,num_val=100,num_test=100):
    from torch_geometric import transforms as T
    transform = T.RandomNodeSplit(split="train_rest",num_val=num_val,num_test=num_test)
    return transform(data)

In [29]:
reddit=datasets.TUDataset(root=root,name="REDDIT-BINARY")
reddit.len()

2000

In [30]:
reddit[0]

Data(edge_index=[2, 480], y=[1], num_nodes=218)

In [31]:

def get_mask_ratio(data,ratio,rnd=None):
    """
    固定种子生成mask
    """
    j = 0
    for i in ratio:
        j+=i
    assert j==1 and len(ratio)==3
    # train,test,val
    num_nodes = data.x.shape[0]
    masks = []
    for i in ratio:
        masks.append(int(num_nodes*i))
    if sum(masks)!=num_nodes:
        masks[-1]=num_nodes-masks[0]-masks[1]
    rnd_index = []
    for i in range(num_nodes):
        rnd_index.append(i)
    if rnd:
        random.seed(rnd)
        random.shuffle(rnd_index)
    train_d,test_d,val_d = rnd_index[:masks[0]],rnd_index[masks[0]:masks[0]+masks[1]],rnd_index[masks[0]+masks[1]:]
    print(f"Splitting {num_nodes} nodes into {masks}")
    train_mask= torch.BoolTensor([False]*num_nodes)
    test_mask= torch.BoolTensor([False]*num_nodes)
    val_mask = torch.BoolTensor([False]*num_nodes)
    for i in train_d:
        train_mask[i] = True
    for i in test_d:
        test_mask[i] = True
    for i in val_d:
        val_mask[i] = True
    return (train_mask,test_mask,val_mask)
  

def get_mask_nums(data,nums,rnd=None):
    """
    固定种子生成mask
    """
    # train,test,val
    num_nodes = data.x.shape[0]
    masks = nums
    rnd_index = []
    for i in range(num_nodes):
        rnd_index.append(i)
    if rnd:
        random.seed(rnd)
        random.shuffle(rnd_index)
    train_d,test_d,val_d = rnd_index[:masks[0]],rnd_index[masks[0]:masks[0]+masks[1]],rnd_index[masks[0]+masks[1]:]
    print(f"Splitting {num_nodes} nodes into {masks}")
    train_mask= torch.BoolTensor([False]*num_nodes)
    test_mask= torch.BoolTensor([False]*num_nodes)
    val_mask = torch.BoolTensor([False]*num_nodes)
    for i in train_d:
        train_mask[i] = True
    for i in test_d:
        test_mask[i] = True
    for i in val_d:
        val_mask[i] = True
    return (train_mask,test_mask,val_mask)



In [5]:
class test():
    def __init__(self) -> None:
        self.node = 1
        self.get_node(self.node)
        

    def get_node(self,node):
        node = 'Yes'
        return node

gg = test()
gg.get_node(gg.node)

'Yes'

In [1]:
a=[1,2,3,4,5,6,7,8,9]
a[-2:]

[8, 9]